In [4]:
#os file adminstration
from os import listdir,getcwd
from os.path import isfile, join

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)

# file loading

In [199]:
#obtain directory
dirpath = getcwd()
#input files in list
onlyfiles = [f for f in listdir(dirpath) if isfile(join(dirpath, f))]
print(onlyfiles)
#subset only required csv files that starts with 'r'
csvfiles = [file for file in  onlyfiles if file[0] == 'r' ]
csvfiles

['api2_balance_novalues.csv', 'api2_nonull_plus_balanced.csv', 'api2_round1_nonull.csv', 'api2_round1_withnull - Copy.csv', 'api2_round1_withnull.csv', 'balance_done.csv', 'cleaned.csv', 'combine cleaned and gps.ipynb', 'combined.ipynb', 'conversion.csv', 'GPS retrieval.ipynb', 'hdb.zip', 'hdb_done.csv', 'metadata-resale-flat-prices.txt', 'mrt.csv', 'mrt_done.csv', 'null_df10.csv', 'resale-flat-prices-based-on-approval-date-1990-1999.csv', 'resale-flat-prices-based-on-approval-date-2000-feb-2012.csv', 'resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv', 'resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv', 'resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv']


['resale-flat-prices-based-on-approval-date-1990-1999.csv',
 'resale-flat-prices-based-on-approval-date-2000-feb-2012.csv',
 'resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv',
 'resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv',
 'resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv']

In [200]:
# concat all the files, row wise 
# drop 'lease_commence_date' as we can calculate that later . 
# 99years - (current year - lease start)
df=pd.DataFrame()
for csvfile in csvfiles:
    temp = pd.read_csv(csvfile)
    try:
        temp.drop(columns=['remaining_lease'],inplace=True)
    except:
        pass
    df=pd.concat([df, temp], axis=0,ignore_index=True)

## data cleaning

In [201]:
#insert a dash in between 'MULTI GENERATION'
df.loc[df['flat_type'] == 'MULTI GENERATION', 'flat_type'] = 'MULTI-GENERATION'

In [202]:
# convert to datetime 
df['date']=pd.to_datetime(df['month'])
# extract year and month from 'date'
df['year_sold']=df['date'].dt.year
df['month_sold']=df['date'].dt.month
df['remaining_lease'] = df['lease_commence_date'] + 99 - df['year_sold']
df.remaining_lease.unique()

array([ 86,  85,  87,  88,  93,  89,  94,  90,  91,  95,  81,  92,  82,
        78,  84,  80,  83,  76,  79,  77,  97,  96,  98,  75, 100,  99,
        74,  73,  72, 101,  71,  70,  69,  68,  67,  66,  65,  64,  63,
        62,  61,  60,  59,  58,  57,  56,  55,  54,  53,  52,  51,  50,
        49,  48,  47,  46], dtype=int64)

In [203]:
df['flat_type']

0            1 ROOM
1            1 ROOM
2            1 ROOM
3            1 ROOM
4            3 ROOM
            ...    
810790       5 ROOM
810791    EXECUTIVE
810792    EXECUTIVE
810793    EXECUTIVE
810794    EXECUTIVE
Name: flat_type, Length: 810795, dtype: object

In [204]:
df.flat_model.unique()

array(['IMPROVED', 'NEW GENERATION', 'MODEL A', 'STANDARD', 'SIMPLIFIED',
       'MODEL A-MAISONETTE', 'APARTMENT', 'MAISONETTE', 'TERRACE',
       '2-ROOM', 'IMPROVED-MAISONETTE', 'MULTI GENERATION',
       'PREMIUM APARTMENT', 'Improved', 'New Generation', 'Model A',
       'Standard', 'Apartment', 'Simplified', 'Model A-Maisonette',
       'Maisonette', 'Multi Generation', 'Adjoined flat',
       'Premium Apartment', 'Terrace', 'Improved-Maisonette',
       'Premium Maisonette', '2-room', 'Model A2', 'Type S1', 'Type S2',
       'DBSS', 'Premium Apartment Loft'], dtype=object)

In [205]:
df.flat_type.unique()

array(['1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE',
       'MULTI-GENERATION'], dtype=object)

In [206]:
df.floor_area_sqm

0          31.0
1          31.0
2          31.0
3          31.0
4          73.0
          ...  
810790    122.0
810791    146.0
810792    164.0
810793    152.0
810794    146.0
Name: floor_area_sqm, Length: 810795, dtype: float64

In [207]:
#change all to lower as there are mixtures of upper and lower
df['flat_model'] = df['flat_model'].str.lower()

In [208]:
#convert storey_range to mean ( example '10 TO 12' convert to 11)
df['storey_range'].str.split(' ').apply(lambda x: (int(x[0]) + int(x[2]))/2)

0         11.0
1          5.0
2         11.0
3          8.0
4          5.0
          ... 
810790    11.0
810791    11.0
810792     8.0
810793     8.0
810794     5.0
Name: storey_range, Length: 810795, dtype: float64

In [209]:
#write to csv for streamlit
df.to_csv('cleaned.csv',index=False)

## Feature engineering GPS

In [13]:
from geopy import Nominatim
from geopy.extra.rate_limiter import RateLimiter

## Method 1 : Use GeoPy library

In [14]:
add_list1 = ["raffles place mrt"]
latitude_list = []
longitude_list = []
for i in add_list1:
    geolocator = Nominatim(user_agent="myGeocoder")
    # convenient function to delay between geocoding calls
    # delay to prevent drop outs
    try:
        location = geolocator.geocode(i,timeout=5)
        latitude_list.append(location.latitude)
        longitude_list.append(location.longitude)
    except:
        latitude_list.append(None)
        longitude_list.append(None)
print(location)

Raffles Place MRT Exit G, Chulia Street, Downtown Core, Singapore, Central, 048616, Singapore


## Method 2 : Using OneMap API

In [15]:
import json
import requests
add = "raffles place mrt"
url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+add
response = requests.get(url)
data = json.loads(response.text) 
data

{'found': 17,
 'totalNumPages': 2,
 'pageNum': 1,
 'results': [{'SEARCHVAL': 'RAFFLES PLACE MRT STATION EXIT D',
   'BLK_NO': '',
   'ROAD_NAME': 'NIL',
   'BUILDING': 'RAFFLES PLACE MRT STATION EXIT D',
   'ADDRESS': 'RAFFLES PLACE MRT STATION EXIT D',
   'POSTAL': '00',
   'X': '30007.10602',
   'Y': '29564.07004',
   'LATITUDE': '1.283641391',
   'LONGITUDE': '103.8513532',
   'LONGTITUDE': '103.8513532'},
  {'SEARCHVAL': 'RAFFLES PLACE MRT STATION EXIT E',
   'BLK_NO': '',
   'ROAD_NAME': 'NIL',
   'BUILDING': 'RAFFLES PLACE MRT STATION EXIT E',
   'ADDRESS': 'RAFFLES PLACE MRT STATION EXIT E',
   'POSTAL': '00',
   'X': '30025.95705',
   'Y': '29516.54412',
   'LATITUDE': '1.283211582',
   'LONGITUDE': '103.8515226',
   'LONGTITUDE': '103.8515226'},
  {'SEARCHVAL': 'RAFFLES PLACE MRT STATION EXIT J',
   'BLK_NO': '',
   'ROAD_NAME': 'NIL',
   'BUILDING': 'RAFFLES PLACE MRT STATION EXIT J',
   'ADDRESS': 'RAFFLES PLACE MRT STATION EXIT J',
   'POSTAL': '00',
   'X': '30000.37445',


Based on our trials, we will be using primarily OneMAP API as it's faster and presumely should be more accurate with respect to GeoPY. Should there be any issue with the scraping with OneMAP API, the fallback is to use GeoPY and manual method if the remaining query is insignificant

## Scraping MRT using OneMap

https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations


In [16]:
# we convert mrt names to a csv file
df_mrt = pd.read_csv('mrt.csv')
df_mrt.head()

,MRT
0,jurong east mrt
1,bukit batok mrt
2,bukit gombak mrt
3,choa chu kang mrt
4,yew tee mrt


In [17]:
df_mrt.shape

(121, 1)

In [18]:
import json
import requests

def get_gps(df,col):
    '''
    This function will obtain the latitude and longitude using oneMAP api
    
    Keyword arguments:
    df : df name
    col : name of the column to retrieve
    '''
    locations = list(df[col])
    latitude_list = []
    longitude_list = []
    for location in locations:
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+location
        response = requests.get(url)
        data = json.loads(response.text) 
        try:
            latitude_list.append(data['results'][0]['LATITUDE'])
            longitude_list.append(data['results'][0]['LONGITUDE'])
          
        except:
            latitude_list.append(None)
            longitude_list.append(None)
    
    df["latitude"] = latitude_list
    df["longitude"] = longitude_list
    
    return df

In [ ]:
# obtain MRT GPS
df_mrt = get_gps(df_mrt,'MRT')
df_mrt.head()
df_mrt.to_csv('mrt_done.csv',index=False)

## Scraping HDB address using OneMap

In [214]:
#combine block and street name so that it can be inputted to the API
df['address'] = df['block'].map(str) + " " + df['street_name']
df["address"]=df["address"].str.lower()
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,date,year_sold,month_sold,remaining_lease,address
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,improved,1977,9000.0,1990-01-01,1990,1,86,309 ang mo kio ave 1
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,improved,1977,6000.0,1990-01-01,1990,1,86,309 ang mo kio ave 1
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,improved,1977,8000.0,1990-01-01,1990,1,86,309 ang mo kio ave 1
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,improved,1977,6000.0,1990-01-01,1990,1,86,309 ang mo kio ave 1
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,new generation,1976,47200.0,1990-01-01,1990,1,85,216 ang mo kio ave 1


since unique values are only over 9000 vs 810,795 for the whole df,
we will scrape the former and merge back to the original df

In [24]:
print(df.shape[0])
print(len(df.address.unique()))

810795
9143


In [23]:
#obtain GPS for the unique values using OneMAP API
df_unique_address = pd.DataFrame(data =df.address.unique(), columns = ['address'] )
df_unique_address = get_gps(df_unique_address,'address')

In [25]:
#backup this file as we do not want to re-run the scraping again
df_unique_address.to_csv('data/unique_address.csv',index=False)

## Use GeoPY to obtain the rest of the locations

In [76]:
#134 rows that we are not able to obtain GPS
df_unique_address['latitude'].isnull().sum()

134

In [77]:
df_balance_null = df_unique_address.loc[df_unique_address['latitude'].isnull(),:]
df_unique_address.loc[df_unique_address['latitude'].isnull(),:].head()

,address,latitude,longitude
0,309 ang mo kio ave 1,None,None
9,247 ang mo kio ave 3,None,None
11,252 ang mo kio ave 4,None,None
48,455 ang mo kio ave 10,None,None
262,4 hillview ave,None,None


In [79]:
# we need to convert the shortform ave to avenue as GeoPY is not as versatle as OneMAP
df_balance_null_split= df_balance_null["address"].str.split(expand=True)
df_balance_null_split.head()

,0,1,2,3,4,5
0,309,ang,mo,kio,ave,1
9,247,ang,mo,kio,ave,3
11,252,ang,mo,kio,ave,4
48,455,ang,mo,kio,ave,10
262,4,hillview,ave,None,None,None


In [80]:
# the conversion that we will be doing
compare = pd.read_csv("conversion.csv")
compare['before']= compare['before'].str.lower()
compare['after']= compare['after'].str.lower()
compare.head()

,before,after
0,a’space,aerospace
1,ably,assembly
2,ac,academy
3,acad,academy
4,adm,administration


In [81]:
rename_dict = compare.set_index('before').to_dict()['after']

In [82]:
# replace before and after ( short form to original )
df_balance_null_split =df_balance_null_split.replace(rename_dict)

In [83]:
df_balance_null_split.iloc[1,:]

0       247
1       ang
2        mo
3       kio
4    avenue
5         3
Name: 9, dtype: object

In [84]:
# we include singapore as the suffix just in case GeoPY is not versatile
# there are instances whereby there are similar location around the world
df_balance_null_split.apply(lambda x: x.str.cat(sep=' ') + ' singapore', axis=1)

0        309 ang mo kio avenue 1 singapore
9        247 ang mo kio avenue 3 singapore
11       252 ang mo kio avenue 4 singapore
48      455 ang mo kio avenue 10 singapore
262            4 hillview avenue singapore
                       ...                
3606    7 seletar west farmway 6 singapore
3739      12 saint george's road singapore
4922       19 kampong bahru hill singapore
5299    6 seletar west farmway 6 singapore
6157      4b saint george's lane singapore
Length: 134, dtype: object

In [85]:
df_balance_null.loc[:,'address'] = df_balance_null_split.apply(lambda x: x.str.cat(sep=' ') + str(' singapore'), axis=1)

C:\Users\andre\AppData\Local\Continuum\anaconda3\envs\streamprophet37\lib\site-packages\pandas\core\indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [227]:
#add_list = list(df_balance_null.address)

def get_GPS_geoPY(df,col):
    '''
    This function will obtain the latitude and longitude using GeoPY
    
    Keyword arguments:
    df : df name
    col : name of the column to retrieve
    '''
    
    latitude_list = []
    longitude_list = []
    for i in list(df[col]) :
        geolocator = Nominatim(user_agent="myGeocoder")
        # convenient function to delay between geocoding calls
        # delay to prevent drop outs
        try:
            location = geolocator.geocode(i,timeout=3)
            latitude_list.append(location.latitude)
            longitude_list.append(location.longitude)
        except:
            latitude_list.append(None)
            longitude_list.append(None)
    df.loc[:,"latitude"] = latitude_list
    df.loc[:,"longitude"] = longitude_list
    return df
    
    #add_df1.to_csv("add_df1.csv")

In [90]:
df_balance_null = get_GPS_geoPY(df_balance_null,'address')

C:\Users\andre\AppData\Local\Continuum\anaconda3\envs\streamprophet37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\andre\AppData\Local\Continuum\anaconda3\envs\streamprophet37\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [91]:
# we will manually input the values since they are only 14 values 
# with respect to initital of 9143
df_balance_null.isnull().sum()

address       0
latitude     14
longitude    14
dtype: int64

In [122]:
df_balance_null[df_balance_null['latitude'].isnull()]

,address,latitude,longitude
302,33 taman ho swee singapore,NaN,NaN
340,18 kampong bahru hill singapore,NaN,NaN
486,1 jalan pasar baru singapore,NaN,NaN
1220,31 taman ho swee singapore,NaN,NaN
1330,5 yung ping road singapore,NaN,NaN
1331,6 yung ping road singapore,NaN,NaN
1598,22 kampong bahru hill singapore,NaN,NaN
1925,51 nile road singapore,NaN,NaN
1926,54 nile road singapore,NaN,NaN
1927,24 jalan membina barat singapore,NaN,NaN


In [ ]:
# write to CSV , manually fill in and re-load

In [124]:
df_balance_null.to_csv('data/df_balance_null.csv',index=False)

In [144]:
df_balance_completed = pd.read_csv('data/df_balance_null.csv')

## Merge back to df_unique 

In [158]:
df_unique_address.shape

(9143, 3)

In [159]:
df_unique_address.loc[~df_unique_address['latitude'].isnull(),:].shape

(9009, 3)

In [157]:
df_unique_address.loc[df_unique_address['latitude'].isnull(),:].shape

(134, 3)

In [160]:
df_balance_completed.shape

(134, 3)

In [183]:
df_unique_null =df_unique_address.loc[df_unique_address['latitude'].isnull(),:].reset_index(drop=True)
df_balance_completed.reset_index(drop=True)

In [186]:
#merge the df_balance_completed 134 values to the original 9143 df_unique
df_unique_null.loc[:,'latitude']=df_balance_completed['latitude']
df_unique_null.loc[:,'longitude']=df_balance_completed['longitude']

In [188]:
df_unique_null.isnull().sum()

address      0
latitude     0
longitude    0
dtype: int64

In [228]:
#subset to 9009 rows 
df_unique = df_unique_address.loc[~df_unique_address['latitude'].isnull(),:].reset_index(drop=True)

In [229]:
#combine df_unique(9009 rows) and df_unique (134 rows)
frames = [df_unique, df_unique_null]
df_unique = pd.concat(frames).reset_index(drop=True)
df_unique.head()

,address,latitude,longitude
0,216 ang mo kio ave 1,1.366196788,103.841505
1,211 ang mo kio ave 3,1.3691969659999998,103.8416666
2,202 ang mo kio ave 3,1.36844644,103.8445163
3,235 ang mo kio ave 3,1.366823609,103.8364912
4,232 ang mo kio ave 3,1.3683460680000001,103.83719599999999


In [230]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,date,year_sold,month_sold,remaining_lease,address,latitude,longitude
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,improved,1977,9000.0,1990-01-01,1990,1,86,309 ang mo kio ave 1,1.36605,103.837
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,improved,1977,6000.0,1990-01-01,1990,1,86,309 ang mo kio ave 1,1.36605,103.837
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,improved,1977,8000.0,1990-01-01,1990,1,86,309 ang mo kio ave 1,1.36605,103.837
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,improved,1977,6000.0,1990-01-01,1990,1,86,309 ang mo kio ave 1,1.36605,103.837
4,1990-02,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,improved,1977,8000.0,1990-02-01,1990,2,86,309 ang mo kio ave 1,1.36605,103.837


In [222]:
df=df.merge(df_unique,on='address')

In [223]:
df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
date                   0
year_sold              0
month_sold             0
remaining_lease        0
address                0
latitude               0
longitude              0
dtype: int64

In [224]:
# Write to CSV for GPS completed
df.to_csv('data/df_gps.csv',index=False)